In [1]:
from osm_integration import OSMManager
from models.adult import Adult
from datetime import datetime
import folium

In [2]:
# Define the center point and radius for OSMManager
center_point = (48.8588897, 2.3200410217200766)  # Example: Ho Chi Minh City center
radius = 5000  # 5 km radius

# Initialize OSMManager
osm_manager = OSMManager(center_point, radius, network_type="drive")

Loading graph from cache: graph_cache/6bf208f59a960e253819f00601a6bdbd.pkl
Scanning for residential...
Scanning for residential near (48.8588897, 2.3200410217200766) within 5000 meters...
Found 97 residential.
Scanning for parks...
Scanning for parks near (48.8588897, 2.3200410217200766) within 5000 meters...
Found 782 parks.
Scanning for schools...
Scanning for schools near (48.8588897, 2.3200410217200766) within 5000 meters...
Found 1319 schools.
Scanning for workplaces...
Scanning for workplaces near (48.8588897, 2.3200410217200766) within 5000 meters...
Found 4718 workplaces.
Scanning for markets...
Scanning for markets near (48.8588897, 2.3200410217200766) within 5000 meters...
Found 714 markets.
Scanning for healthcare...
Scanning for healthcare near (48.8588897, 2.3200410217200766) within 5000 meters...
Found 1119 healthcare.
Scanning for play_areas...
Scanning for play_areas near (48.8588897, 2.3200410217200766) within 5000 meters...
Found 693 play_areas.
Scanning for gyms...
S

In [4]:
# Create an adult person
adult = Adult(unique_id=1, person_type="adult", speed=11.4, osm_manager=osm_manager)

Trajectory built from (48.89174486132547, 2.3023119972446877) to (48.856013427576144, 2.2678282530846796) with 68 nodes.
Trajectory built from (48.856013427576144, 2.2678282530846796) to (48.8922190141131, 2.2982799744212903) with 52 nodes.
Trajectory built from (48.8922190141131, 2.2982799744212903) to (48.856013427576144, 2.2678282530846796) with 53 nodes.
Trajectory built from (48.856013427576144, 2.2678282530846796) to (48.8928314, 2.2977128) with 52 nodes.
Trajectory built from (48.8928314, 2.2977128) to (48.89174486132547, 2.3023119972446877) with 7 nodes.


In [5]:
# Print the assigned waypoints
print("\n--- Assigned Waypoints ---")
for waypoint, coords in adult.waypoints.items():
    if coords:
        print(f"{waypoint.capitalize()}: {coords}")


--- Assigned Waypoints ---
Home: (48.89174486132547, 2.3023119972446877)
Workplace: (48.856013427576144, 2.2678282530846796)
Gym: (48.8928314, 2.2977128)
Market: (48.8922190141131, 2.2982799744212903)


In [6]:
# Print the schedule
print("\n--- Schedule ---")
for movement in adult.schedule:
    print(f"{movement['start_waypoint']} → {movement['end_waypoint']} at {movement['start_time']}")


--- Schedule ---
home → workplace at 07:00:00
workplace → market at 12:00:00
market → workplace at 13:00:00
workplace → gym at 18:00:00
gym → home at 20:00:00


In [7]:
# Print the detailed schedule with trajectories
print("\n--- Detailed Schedule ---")
for movement in adult.detail_schedule:
    print(f"{movement['start_waypoint']} → {movement['end_waypoint']}")
    print(f"  Start Time: {movement['start_time']}")
    print(f"  Arrival Time: {movement['arrival_time']}")
    print(f"  Distance: {movement['distance_m']} meters")
    print(f"  Travel Time: {movement['travel_time_s']} seconds")
    print(f"  Route Nodes: {movement['route_nodes']}")


--- Detailed Schedule ---
home → workplace
  Start Time: 07:00:00
  Arrival Time: 07:07:44.356078
  Distance: 5293.659290148476 meters
  Travel Time: 464.3560780831997 seconds
  Route Nodes: [9014025575, 442321, 27323304, 9814086387, 9814086388, 9020074055, 9020073970, 9020074047, 27288863, 9738056575, 1987578932, 9844132326, 1987617275, 1987617271, 27233893, 27234076, 27234190, 5410385401, 27234191, 27234192, 27234196, 273416715, 27235462, 27235445, 21661018, 273415577, 21660970, 12009276962, 27286711, 27286712, 27286715, 21660281, 661218, 368319, 368318, 21292970, 1962530964, 368313, 368310, 21660044, 34816758, 34816573, 25624001, 34816775, 34816475, 34816542, 34816544, 34816546, 34816547, 34816507, 1761442006, 34817741, 34816712, 34816999, 34816433, 11025071863, 34817011, 34817010, 34817009, 34817006, 34817789, 34817793, 11873654231, 11026499124, 4718747866, 34818596, 34818913, 34818912]
workplace → market
  Start Time: 12:00:00
  Arrival Time: 12:08:55.305158
  Distance: 6102.4788

In [16]:
# Simulate position at a specific time
current_time = datetime.now().replace(hour=7, minute=10)  # Example time
position, log = adult.get_position_at_time(current_time)

print("\n--- Simulated Position ---")
print(f"Time: {current_time}")
print(f"Position (latitude, longitude): {position}")

Checking position at time: 07:10:25.184406
Movement 0: home → workplace
    Start time: 07:00:00, Arrival time: 07:07:44.356078
Movement 1: workplace → market
    Start time: 12:00:00, Arrival time: 12:08:55.305158
    Current time is before this movement. Returning start waypoint: workplace

--- Simulated Position ---
Time: 2025-01-14 07:10:25.184406
Position (latitude, longitude): (48.856013427576144, 2.2678282530846796)


In [17]:
# Create a Folium map centered at the person's home
home_coords = adult.waypoints["home"]
folium_map = folium.Map(location=home_coords, zoom_start=13)

# Add waypoints to the map
for waypoint, coords in adult.waypoints.items():
    if coords:
        folium.Marker(
            location=coords, 
            popup=waypoint.capitalize(), 
            icon=folium.Icon(color="blue")
        ).add_to(folium_map)

# Add trajectories to the map
for movement in adult.detail_schedule:
    route_nodes = movement["route_nodes"]
    route_coords = [(osm_manager.nodes.loc[node, "y"], osm_manager.nodes.loc[node, "x"]) for node in route_nodes]
    
    # Draw the route as a polyline
    folium.PolyLine(route_coords, color="red", weight=2.5, opacity=0.8).add_to(folium_map)


# Add current position to the map
folium.Marker(
    location=position,
    popup=f"Current Position at {current_time.time()}",
    icon=folium.Icon(color="green", icon="info-sign"),
).add_to(folium_map)

In [18]:
# Save the map to an HTML file
map_filename = "map/person_movement_map.html"
folium_map.save(map_filename)
print(f"Map saved to {map_filename}")


Map saved to map/person_movement_map.html
